In [ ]:
import os
import pandas as pd

from fuzzy_matching import *


In [ ]:

file_path = '../models/cleaned_atr.csv'

if not os.path.exists(file_path):
    print(f"File not found: {file_path}")

df = pd.read_csv(file_path)
df.head()

df['Destination'] = df['Destination'].fillna("nan").astype('string')
df['AisSourcen'] = df['AisSourcen'].fillna("nan").astype('string')

df['StartTime'] = pd.to_datetime(df['StartTime'], utc=True)
df['EndTime'] = pd.to_datetime(df['EndTime'], utc=True)
df['time'] = pd.to_datetime(df['time'], utc=True)  # Appears to have timezone info (+01:00)

df['StartPort'] = df['StartPort'].astype('string').astype('category')
df['EndPort'] = df['EndPort'].astype('string').astype('category')

df.dtypes

## ==================================== Step 1
Deal with Inconsistent Records (We can have different formats representing the same thing) if they are there.
And convert columns to categorical where appropriate.
## ====================================

In [ ]:
def check_unique_values(df):
    """Check unique values in each column of the DataFrame."""
    col_un = {}
    for col in df.columns:
        clean_series = df[col].dropna()
        nunique = clean_series.nunique()
        col_un[col] = nunique
    return col_un

unique_values_before = check_unique_values(df)
unique_values_before

Column: StartPort, Unique values: 2 - seems correct
Column: EndPort, Unique values: 2 - seems correct

Column: Destination, Unique values: 140 - weird
Column: AisSourcen, Unique values: 224 - should check if it is correct

In [ ]:
# Case normalization
text_columns = df.select_dtypes(include=['string']).columns
for col in text_columns:
    df[col] = df[col].str.upper()  # Ensure string type and uppercase

unique_values_after = check_unique_values(df)

changed_columns = list(filter(lambda col: unique_values_after[col] != unique_values_before[col], df.columns))
df[changed_columns].dropna().nunique() # Check how many unique values are there after case normalization

Case normalization has changed the unique values in the following columns: Destination
It seems there are still inconsistencies in the Destination column, so we will need to clean it further.

In [ ]:
df['Destination'].unique() #See some examples of the Destination column

In [ ]:
df['Destination'] = df['Destination'].where(
        df['Destination'].str.contains(r'[A-Za-z]', na=False),
        "NAN"
    ) # Atle ast one alphabetic

df['Destination'] = df['Destination'].apply(
    lambda x: "NAN" if re.match(r'^[A-Z]{2}$', str(x)) else x
) #only country code

def find_values_with_special_chars(df):
    """Find values with special characters in the 'Destination' column."""
    return [
        value for value in df['Destination'].unique()
        if re.search(r'[^A-Za-z0-9]', str(value))
    ]


In [ ]:
dest_before = find_values_with_special_chars(df)
print(len(dest_before), "unique values before cleaning with special characters")
dest_before

## ==================================== Step 2
We can see that we have different formats representing the same thing, like 'HAMBURG' and 'DEHAM' ext.
The data is incredibly messy, we need to handle country codes, special characters, and different formats.

1. Some have country codes
2. Some contain starting port too
3. Some contain type of facilities (e.g., 'ELBE.RC', 'BREMERHAVEN.VIA.NOK')

start > destination
## ====================================

In [ ]:
# Clean all data
df['Destination'] = df['Destination'].apply(clean_destination)
df['Destination'].unique()

In [ ]:
# Create mask for rows containing '>'
mask = df['Destination'].str.contains('>', na=False)

# Initialize columns (if not already done)
df['start_fr_dest'] = None
# df['cleaned_destination'] = df['Destination'].copy()

# Split and assign values safely
df.loc[mask, 'start_fr_dest'] = df.loc[mask, 'Destination'].str.split('>').str[0]
df.loc[mask, 'Destination'] = df.loc[mask, 'Destination'].str.split('>').str[1]
df

#### **I CREATED CUSTOM FILE WITH FUNCTION WE WILL USE DOWN**

df_recombined['Destination'].unique()
Now we have somewhat cleaned Destination column, but we still have some inconsistencies. That the same ports have different names.
I didn't find quicker way to do it, rather than manually checking the names and creating a list of names that represent the same port.
We will only a bit automize this process, by using fuzzy matching to find similar names.
And extracting all ports from UpdatedPub150.csv file, which contains ports and their countries.
[link](https://msi.nga.mil/Publications/WPI)

In [ ]:
# ports_cvs = '../data/UpdatedPub150.csv'
# port_df = pd.read_csv(ports_cvs)
# save_filtered_ports(port_df, country_name='Poland')
# save_filtered_ports(port_df, country_name='Germany')
# save_filtered_ports(port_df, country_name='Lithuania')
# save_filtered_ports(port_df, country_name='Sweden')

In [ ]:
unique_dests = df['Destination'].unique().tolist()

# Find matches with threshold of 85
matches = find_fuzzy_matches(unique_dests, threshold=75, show_progress=True)

# Print results grouped by similarity
print_fuzzy_matches(matches, min_score=75, group_similar=True)

From this I will manually create a list of names that represent the same port, that do not have 100 match.
As they can be incorrectly matched, and it would be better to do it manually.

https://www.marinetraffic.com/en/ais/details/ports/347?name=HALMSTAD&country=Sweden


In [ ]:
df['start_fr_dest'].unique()

In [ ]:
name_german = {
    'DEHAM': ["HAMBURG", "HAMBUG", "HH", "HAM"],
    'DEBRV': ["BREMERHAVEN", "BREMENHAVEN", "BRV", "DEBHV", "BHV"],
    'DEBRE': ["BREMEN", "BRE"],
    'DEKEL': ["KIEL", "KEL"],
    'DEHAM.FINKENWERDER': ["FINKENWERDER", "FINKENWERD"],
    'DEHAM.BLEXEN': ["BLEXEN"],
    'DESTA': ["STADE", "STAD", "STA"],
    'DEBRB' : ["BRUNSBUETTEL", "BRUNSBUETT", "BRB"],
    'DEHAM.ELBE': ["ELBE"],
    'DEVTT': ["HIDDENSEE", "VTT"],
    'DEWVN': ["WILHELMSHAVEN", "WVN"],
    'country': ["DE"]
}

name_poland = {
    'PLGDN': ["GDANSK", "GDANK", "GDN"],
    'PLGDY': ["GDYNIA", "GYDNIA", "GYDINIA", "GDY", "GDYNA"],
    'SZCZECIN': ["SZCZECIN", "SZCZECIN", "SZZ"],
     'country': ["PL"]
}

name_lythuania = {
    'LTKLJ': ["KLAIPEDA", "KLJ"],
    'country': ["LT"]
}

name_sweden = {
    'SEHAD': ["HALMSTAD", "HAD"],
    'SENOK': ["NOK"],
    'SEAHU': ["AHUS", "AHU"],
    'country': ["SE"]
}

name_russia = {
        'RUKGD': ["KALININGRAD", "KALININGRAD", "KAL"],
        'country': ["RU"]
}

name_denmark = {
    'DKKOB': ["KOBENHAVN", "COPENHAGEN", "COPENHAGUE", "CPH"],
    'country': ["DK"]
}

name_finland = {
    'FIHKO': ["HANKO", "HKO"],
    'country': ["FI"]
}

name_belgium = {
    "BEANR": ["BEANR", "ANR"],
    'country': ["BE"]
}
full_dict = [name_german, name_poland, name_lythuania, name_sweden, name_russia, name_finland, name_belgium]

def replace_with_key(df, column, name_variants):
    df[column] = df[column].apply(lambda x: match_names(x, name_variants))
    return df

In [ ]:
df = replace_with_key(df, 'Destination', full_dict)
df = replace_with_key(df, 'start_fr_dest', full_dict)
df[['Destination']].reset_index().drop_duplicates(subset=['Destination'])

In [ ]:
df[['start_fr_dest']].reset_index().drop_duplicates(subset=['start_fr_dest'])

361854,387562,NORDEN.DEHAM
Theoretically, I can switch places for it, but I am lazy

Do we need to split it too? like debrv.via.nok?


In [ ]:
save_path = '../data/normalized_destinations.csv'
df.to_csv(save_path, index=False)